# Budget allocation proposal

### Pseudo-Revenue, First-Revenue assumption
### Regressions

In [1]:
import pandas as pd
import numpy as np
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import partial

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%matplotlib inline
plt.rcParams["figure.figsize"] = [10, 5]

validDf = pd.read_csv("../data/valid_dataset_minimal.csv").drop(columns=["Unnamed: 0"])
campaigns = validDf["Campaign"].unique()
validDf["Date"] = pd.to_datetime(validDf["Date"])
# print(validDf.head(10))
print(validDf.dtypes)

Date                     datetime64[ns]
Source / Medium                  object
Campaign                         object
Device Category                  object
Users                             int64
Sessions                          int64
Bounces                           int64
Transactions                      int64
Cost                            float64
Revenue                         float64
Product Detail Views              int64
Product Adds To Cart              int64
Product Checkouts                 int64
Session Duration                 object
Total Unique Searches             int64
Pageviews                        object
New Users                         int64
Session Duration (s)            float64
Medium                           object
dtype: object


## 0. Dataset creation

### 0.1 Bucketing

In [14]:
def getCampaignFirstRevenue(df):
    df["Bucket Index"] = 0
    # This holds the indexes (starting from 0) in the df where we've got positive revenues
    indexesR = df.loc[df["Revenue"] > 0, "Revenue"].index.values
    # The same, but shifted to the left with one value and starting with -1
    indexesL = np.array([-1, *indexesR[0 : -1]])
    nRows = len(indexesL)

    bucketIndex = -1
    for i in range(nRows):
        l, r = indexesL[i], indexesR[i]
        where = (df.index > l) & (df.index <= r)
        items = df[where]
        # This merges conseccutive buckets if some of them have sum of costs 0, as that'd be useless data
        # Instead, we assume that multiple consecutive revenues must be merged together and divided to first
        if items["Cost"].sum() > 0:
            bucketIndex += 1
        df.loc[where, "Bucket Index"] = bucketIndex
    
    items = df[df.index > indexesR[-1]]
    assert len(items) == 0
    return df

# Return a new column called "bucket index", which represents on what (Cost, Revenue) bucket each entry goes
# This is unique for all campaigns, so bucket index == 1 for Campaign "A", is a different bucket from bucket
#  index == 1 for Campaign "B". This column is used to train on the same logical data (Bucket, Revenue), and
#  we don't get Costs from same bucket both in train and validation set.
# Other assumptions (like weekly assumption) can update this indexes as they will to provide other logic of
#  implementation for the dataset split.
def getDataFirstRevenue(df):
    newDf = pd.DataFrame(columns=df.columns)
    campaigns = df["Campaign"].unique()
    for campaign in campaigns:
        dfCampaign = df[df["Campaign"] == campaign].reset_index(drop=True)
        newDfCampaign = getCampaignFirstRevenue(dfCampaign)
        numBuckets = len(newDfCampaign["Bucket Index"].unique())
        print("Campaign '%s' has %d (Cost, Revenue) buckets." % (campaign, numBuckets), end="")
        if numBuckets < 20:
            print(" Below 20, skipping.")
            continue

        revenues = newDfCampaign["Revenue"]
        # print(campaign, np.percentile(revenues, [0.1, 1, 10, 25, 50, 75, 90, 99, 99.5, 99.9, 100]))
        # Clip the revenues to top 99.5%, to eliminate outliers.
        Max = newDfCampaign["Revenue"].max()
        Top99_5 = np.percentile(newDfCampaign["Revenue"], 99.5)
        print(" Pruning max revenue from %2.3f to %2.3f" % (Max, Top99_5))
        newDfCampaign["Revenue"] = np.clip(newDfCampaign["Revenue"], 0, Top99_5)
        newDf = pd.concat([newDf, newDfCampaign], sort=False)      
    return newDf

firstRevenueDf = getDataFirstRevenue(validDf)
print("___________________________________")
print("Original df: %s. Training df: %s" % (validDf.shape, firstRevenueDf.shape))

Campaign 'AW - Accessories' has 452 (Cost, Revenue) buckets. Pruning max revenue from 1523.180 to 806.975
Campaign 'AW - Apparel' has 138 (Cost, Revenue) buckets. Pruning max revenue from 1623.750 to 132.623
Campaign 'Remarketing' has 5 (Cost, Revenue) buckets. Below 20, skipping.
Campaign 'AW - Dynamic Search Ads Whole Site' has 382 (Cost, Revenue) buckets. Pruning max revenue from 2224.430 to 828.886
Campaign 'AW - Bags' has 52 (Cost, Revenue) buckets. Pruning max revenue from 284.580 to 184.789
Campaign 'AW - Google Brand' has 56 (Cost, Revenue) buckets. Pruning max revenue from 627.490 to 145.900
Campaign 'AW - Office' has 36 (Cost, Revenue) buckets. Pruning max revenue from 1857.000 to 170.745
Campaign 'AW - YouTube' has 5 (Cost, Revenue) buckets. Below 20, skipping.
Campaign 'AW - Drinkware' has 10 (Cost, Revenue) buckets. Below 20, skipping.
Campaign 'AW - YouTube Brand' has 10 (Cost, Revenue) buckets. Below 20, skipping.
Campaign 'All Products' has 109 (Cost, Revenue) buckets. 

### 0.2 Pseudo-Revenue

For Linear Regression we've got the problem that the training data and the testing data is not from the same distribution.

Basically, the problem is that we are training on Data: $X=\sum(Costs)$ and Labels: $t=Revenue$, however, when we are using the model, we are generating from the distribution Data: $X=Cost$, PseudoRevenue: $y=f(X)$.

The problem can be the simplest explained as: $a * f(x_1 + x_2) + b \neq [a * f(x_1) + b] + [a * f(x_2) + b] $.

However, we can always pre-split our data in first-revenue assumption AND linear time assumption (see proposal), basically instead of having $X=\sum_n(Costs)$, $t=Revenue$, we can have $X=[Cost_1, ..., Cost_n]$, $t=[\frac{Cost_1}{Revenue}, ..., \frac{Cost_n}{Revenue}]$.

The second nuance here is that we've got our data split in buckets of $(Cost, Revenue)$. Thus, what we do instead is for each bucket:

$Cost_{B} = [Cost_1, ..., Cost_n]$ and $Revenue_{B} = [Revenue_1, ..., Revenue_m]$. We sum the revenues and the costs independently, to get $C_{\Sigma B} = \sum_{i}^{n}Cost_i$ and $R_{\Sigma B} = \sum_{i}^{m}Revenue_i$. We get the bucket's constant $\alpha_B = \frac{R_{\Sigma B}}{C_{\Sigma B}}$

For statistical model, we had a dataset constant, which was the model's only parameter. Here, we get a bucket constant, which we use to create the pseudo revenue, by multiplying each cost: $Pseudo-Revenue_{B} = [Cost_1 * \alpha_B, ..., Cost_n * \alpha_B]$

If we were to not use a linear time assumption, we'd have a non-linear ponder $C_{\Sigma B} = \sum_{i}^{n}(Cost_i * w_{i,n})$ where $w_{i, n}$ is somehow dependant on current timestamp (i) and final timestamp until last time of this bucket (n). This can be done using a separate model or some other way of detecting it. However, we'll just use a constant $w=1$ and assume all costs are identical to generating the next revenue.

**Example:**
Let a bucket be: $Cost_B=[0, 0, 50, 20, 0, 15]$, $Revenue_B=[30, 100]$. This means that the first revenue (30) was generated by the first two costs alone, so we merged the next bucket as well.

We'll sum them, getting $C_{\Sigma B}=85$ and $R_{\Sigma B}=130$. Then, the bucket constant is: $\alpha_B=130/85=1.529$.

Then, our pseudo-revenues will be: $Pseudo-Revenue_{B} = [0*\alpha_B, 0*\alpha_B, 50*\alpha_B, 20*\alpha_B, 0*\alpha_B, 15*\alpha_B] = [0, 0, 76.45, 30.58, 0, 22.935]$.

These are the values we'll predict for this bucket in our Regressions. We can see that summing them, we get $129.965$, which is almost $130$, the error being caused by truncations.

In [17]:
def f(x):
    sumCost = x["Cost"].sum()
    sumRevenue = x["Revenue"].sum()
    bucketConstant = sumRevenue / sumCost
    return bucketConstant

def getPseudoRevenues(df):
    df = df.copy()
    dfGB = df.groupby(["Campaign", "Bucket Index"]).apply(lambda x : pd.Series({
        "Bucket Constant" : f(x)
    })).reset_index()
    df = pd.merge(df, dfGB, on=["Campaign", "Bucket Index"])
    df["Pseudo Revenue"] = df["Cost"] * df["Bucket Constant"]
    return df

trainDf = getPseudoRevenues(firstRevenueDf)
trainDf["Bucket Index"] = trainDf["Bucket Index"].astype(int)

### 0.2 Get Dataset statistics: Mins/Maxs

In [18]:
def getMinMax(df):
    numericDf = df.select_dtypes(include=[np.number])
    numericDf = numericDf.join(df["Campaign"])
    Mins = numericDf.groupby("Campaign").min().reset_index()
    Maxs = numericDf.groupby("Campaign").max().reset_index()
    return Mins, Maxs

Mins, Maxs = getMinMax(trainDf)
display(Mins)
display(Maxs)

,Campaign,Cost,Revenue,Session Duration (s),Bucket Index,Bucket Constant,Pseudo Revenue
0,AW - Accessories,0.01,0.0,0.0,0,0.037720,0.000532
1,AW - Apparel,0.01,0.0,0.0,0,0.017278,0.000173
2,AW - Bags,0.01,0.0,0.0,0,0.069662,0.000697
3,AW - Dynamic Search Ads Whole Site,0.01,0.0,0.0,0,0.029557,0.003863
4,AW - Google Brand,0.01,0.0,0.0,0,0.033009,0.000330
5,AW - Office,0.01,0.0,0.0,0,0.377298,0.003773
6,All Products,0.01,0.0,0.0,0,0.583503,0.010041


,Campaign,Cost,Revenue,Session Duration (s),Bucket Index,Bucket Constant,Pseudo Revenue
0,AW - Accessories,74.04,806.97485,215881.0,451,1721.739130,698.189290
1,AW - Apparel,45.38,132.62345,215100.0,137,2059.000000,132.623450
2,AW - Bags,35.76,184.78880,214020.0,51,112.769231,184.788800
3,AW - Dynamic Search Ads Whole Site,54.57,828.88560,215940.0,381,112.391137,770.796582
4,AW - Google Brand,35.17,145.90000,215880.0,55,134.000000,144.408180
5,AW - Office,17.52,170.74500,215100.0,35,203.267857,170.745000
6,All Products,9.02,138.10800,214080.0,108,860.500000,108.176744


# 1. Linear Regression (no extra features)

In [27]:
# from ipynb.fs.defs.nb3 import KFold
import torch as tr
import torch.nn as nn

tr.backends.cudnn.deterministic = True
tr.backends.cudnn.benchmark = False

class LinearRegression(nn.Module):
    def __init__(self, numFeatures, useBias=True):
        super(LinearRegression, self).__init__()
        self.useBias = useBias
        self.numFeatures = numFeatures + int(self.useBias)
        tr.manual_seed(42)
        self.W = tr.randn(self.numFeatures, 1).requires_grad_(True)
        self.trained = False

    def prepare(self, X, t):
        assert not self.trained
        assert len(X.shape) == 2 and X.shape[-1] == self.numFeatures - self.useBias
#         assert X.min() >= 0 and X.max() <= 1
        tr.manual_seed(42)
        self.W = tr.randn(self.numFeatures, 1).requires_grad_(True)
        
        X = X.astype(np.float32)
        t = t.astype(np.float32)
        
        # Add bias
        if self.useBias:
            X = np.append(X, np.ones((len(X), 1), dtype=np.float32), axis=-1)
        X = tr.from_numpy(X)
        t = tr.from_numpy(t).unsqueeze(dim=-1)
        return X, t
    
    def fit(self, X, t):
        return self.fit_iter(X, t)
        # TODO
#         X, t = self.prepare(X, t)
#         A = (X * t).mean(dim=0)
#         B = (X**2).mean(dim=0)
#         self.W = (A / B).detach().unsqueeze(dim=-1)
    
    def fit_iter(self, X, t, numIterations=1, lr=0.001):
        X, t = self.prepare(X, t)
        for i in range(numIterations):
            L = self.criterion(X, t)
            L.backward()
            self.W.data -= lr * self.W.grad
            self.W.grad *= 0
        self.trained = True

    def criterion(self, X, t):
        y = tr.mm(X, self.W)
        L = (y - t)**2
        L = L.mean() + ((self.W)**2).mean()
        return L
        
    def predict(self, X):
#         assert self.trained
        X = X.astype(np.float32)
        if self.useBias:
            X = np.append(X, np.ones((len(X), 1), dtype=np.float32), axis=-1)
        X = tr.from_numpy(X)
        y = tr.mm(X, self.W)
        return y.detach().numpy()

In [28]:
def errorL1(y, t):
    return np.abs(y - t).mean()

def trainModelLR(X, t, errorFn, Mappings, modelType, numSplits=5, randomState=42):
    kf = KFold(numSplits=numSplits, randomState=randomState)
    errors = []
    Max = Mappings.max()
    ix = np.arange(Max)
    for trainIx, validationIx in kf.split(Max):
        print(trainIx, )
        model = modelType()
        trainMap, valMap = ix[trainIx], ix[validationIx]
        whereTrain = np.isin(Mappings, trainMap)
        whereVal = np.isin(Mappings, valMap)
        model.fit(X[whereTrain], t[whereTrain])
        y = model.predict(X[whereVal])
        errors.append(errorFn(y, t[whereVal]))

    # We report the mean error on all K-fold splits for robustness
    meanError = np.mean(errors)
    
    # Then, we retrain on all data, for best test/future predictions
    model = modelType()
    model.fit(X, t)
    return model, meanError

def trainAllCampaignsLR(df, modelType, features):
    models, errors = {}, {}
    validCampaigns = df["Campaign"].unique()

    for i, campaign in enumerate(validCampaigns):
        where = df["Campaign"] == campaign
        X = df.loc[where, features].values
        t = df.loc[where, ["Revenue"]].values
        Mappings = df.loc[where, "Bucket Index"].values
        model, error = trainModelLR(X, t, errorL1, Mappings, modelType)
        models[campaign] = model
        errors[campaign] = error
    return models, errors

In [29]:
features = ["Cost"]
LR = partial(LinearRegression, numFeatures=len(features))
models, errors = trainAllCampaignsLR(trainDf, LR, features)

for k in errors:
    print(k, errors[k])

NameError: name 'KFold' is not defined

In [ ]:
# def trainModelLR(X, t, Mappings, modelType, numSplits, randomState):
#     kf = KFold(numSplits=numSplits, randomState=randomState)
#     errors = []
#     Max = Mappings.max()
#     ix = np.arange(Max)
#     for trainIx, validationIx in kf.split(Max):
#         model = modelType()
#         trainMap, valMap = ix[trainIx], ix[validationIx]
#         whereTrain = np.isin(Mappings, trainMap)
#         whereVal = np.isin(Mappings, valMap)
#         model.fit(X[whereTrain], t[whereTrain])
#         y = model.predict(X[whereVal])
#         errors.append(errorL1(y, t[whereVal]))

#     # We report the mean error on all K-fold splits for robustness
#     meanError = np.mean(errors)
    
#     # Then, we retrain on all data, for best test/future predictions
#     model = modelType()
#     model.fit(X, t)
#     return model, meanError

# def trainAllCampaignsLR(df, X, t, modelType, columns, XXs, numSplits, randomState):
#     models, errors = {}, {}
#     validCampaigns = list(X.keys())
#     df = df.copy()

#     for i, campaign in enumerate(validCampaigns):
#         thisT = np.clip(t[campaign], 0, np.percentile(t[campaign], XXs[campaign]))
#         model, error = trainModelLR(X[campaign], thisT, Mappings[campaign], modelType, numSplits, randomState)
#         dfCampaign = df[df["Campaign"] == campaign]
#         pseudo = model.predict(dfCampaign[columns].values)
#         error = dfCampaign["Revenue"].sum() - pseudo.sum()
#         models[campaign] = model
#         errors[campaign] = error
#     return models, errors

# def getOptimalThresholds(df, X, t, modelType, columns, numSplits, randomState):
#     XXs = {}
#     validCampaigns = list(X.keys())
# #     Optimize threshold for all campaigns to remove outliers.
#     for i, campaign in enumerate(validCampaigns):
#         res = []
#         for i in range(100):
#             thisT = np.clip(t[campaign], 0, np.percentile(t[campaign], i))
#             model, error = trainModelLR(X[campaign], thisT, Mappings[campaign], modelType, numSplits, randomState)
#             dfCampaign = df[df["Campaign"] == campaign].copy()
#             dfCampaign["Pseudo Revenue"] = model.predict(dfCampaign[columns])
#             error = np.abs(dfCampaign["Revenue"].sum() - dfCampaign["Pseudo Revenue"].sum())
#             res.append(error)
#         XXs[campaign] = np.argmin(res)
#     return XXs

First, optimize threshold for each campaign, then use best one to get best model.

In [ ]:
from sklearn.linear_model import LinearRegression
LR = partial(LinearRegression, fit_intercept=False, normalize=True)
XXs = getOptimalThresholds(validDf, XLinearRegression, tLinearRegression, LR, columns=["Cost"], numSplits=5, randomState=42)
print(XXs)

In [ ]:
LRModels, LRErrors = trainAllCampaignsLR(validDf, XLinearRegression, tLinearRegression, LR, columns=["Cost"], XXs=XXs, numSplits=5, randomState=42)
display(pd.DataFrame([(x, y) for x, y in zip(LRErrors.keys(), LRErrors.values())], columns=["Campaign", "Best error (L1)"]))

In [ ]:
from ipynb.fs.defs.nb3 import createPseudoRevenues
LRFinalDf = createPseudoRevenues(LRModels, validDf, features=["Cost"])
print(LRFinalDf.shape, LRFinalDf.columns)

In [ ]:
from ipynb.fs.defs.nb3 import plotRevenues
plotRevenues(LRFinalDf)

In [ ]:
from ipynb.fs.defs.nb3 import scatterRevenues
scatterRevenues(LRFinalDf)

In [ ]:
from ipynb.fs.defs.nb3 import getSummedPseudoRevenues
LRFinalDf = getSummedPseudoRevenues(LRFinalDf, features=["Cost"])
print(LRFinalDf.shape, LRFinalDf.columns)

In [ ]:
from ipynb.fs.defs.nb3 import plotSummedRevenues
plotSummedRevenues(LRFinalDf)

In [ ]:
from ipynb.fs.defs.nb3 import scatterSummedRevenues
scatterSummedRevenues(LRFinalDf)

In [ ]:
from ipynb.fs.defs.nb3 import scatterErrorSummedRevenues
scatterErrorSummedRevenues(LRFinalDf)

In [ ]:
from ipynb.fs.defs.nb3 import getOverallError
LROverallError = getOverallError(LRFinalDf)
display(LROverallError)

In [ ]:
from ipynb.fs.defs.nb3 import plotFinalResults
plotFinalResults(LRFinalDf, LROverallError)

### 2.4 Linear regression with more features

(TODO)

#### 2.4.1 Dataset

In [ ]:
features = ["Cost", "Device Category"]
validOneHotDf = pd.get_dummies(validDf[features].reset_index(drop=True))
features = validOneHotDf.columns

validOneHotDf = pd.merge(validDf[["Date", "Campaign", "Revenue"]], validOneHotDf, left_index=True, right_index=True).reset_index(drop=True)
XFeats, t = getDataFirstRevenue(validOneHotDf, features=features)

#### 2.4.1.2 Convert columns to one-hot where possible

In [ ]:
XFeatsLR, tFeatsLR, Mappings = {}, {}, {}

def getDataLR(X, t):
    Mappings, XRes, tRes = [], [], []
    N = len(X)
    for i in range(N):
        cost = X[i][:, 0]
        XRes.extend(list(X[i]))
        tRes.extend(list(t[i] / cost))
        Mappings.extend([i] * len(X[i]))
    return np.array(XRes), np.array(tRes).reshape((-1, 1)), np.array(Mappings)

for k in X:
    XFeatsNoDummy, tFeatsLR[k], Mappings[k] = getDataLR(XFeats[k], t[k])
    
    dummied = []
    numFeatures = XFeatsNoDummy.shape[-1]
    for i in range(numFeatures):
        item = XFeatsNoDummy[:, i]
        try:
            _ = item.astype(np.float32)
            item = item.reshape(-1, 1)
        except Exception as e:
            item = pd.get_dummies(item)
        dummied.append(item)
    XFeatsLR[k] = np.concatenate(dummied, axis=-1)
    print(k, XFeatsLR[k].shape)

In [ ]:
LR = partial(LinearRegression, fit_intercept=True, normalize=True)
XXs = getOptimalThresholds(validOneHotDf, XFeatsLR, tFeatsLR, LR, columns=features, numSplits=5, randomState=42)
print(XXs)

In [ ]:
LRModels, LRErrors = trainAllCampaignsLR(validOneHotDf, XFeatsLR, tFeatsLR, LR, columns=features, XXs=XXs, numSplits=5, randomState=42)
display(pd.DataFrame([(x, y) for x, y in zip(LRErrors.keys(), LRErrors.values())], columns=["Campaign", "Best error (L1)"]))

In [ ]:
LRFinalDf = createPseudoRevenues(LRModels, validOneHotDf, features)
print(LRFinalDf.shape, LRFinalDf.columns)

In [ ]:
plotRevenues(LRFinalDf)

In [ ]:
scatterRevenues(LRFinalDf)

In [ ]:
LRFinalDf = getSummedPseudoRevenues(LRFinalDf, features=features)
print(LRFinalDf.shape, LRFinalDf.columns)

In [ ]:
plotSummedRevenues(LRFinalDf)

In [ ]:
scatterSummedRevenues(LRFinalDf)

In [ ]:
scatterErrorSummedRevenues(LRFinalDf)

In [ ]:
LROverallError = getOverallError(LRFinalDf)
display(LROverallError)

In [ ]:
plotFinalResults(LRFinalDf, LROverallError)